In [1]:
import numpy as np
import rasterio
from osgeo import gdal
from rasterio.windows import Window
driver = gdal.GetDriverByName("GTiff")

## Define Functions to Update Window Extents

In [2]:
def get_window_crop(img,tile_size):
    #this function crops the image with metadata to match inference extents, as per inference cropping process
    #from rasterio docs: Window(col_off, row_off, width, height)
    x = img.width
    y = img.height
    cropx = (round(int(x/tile_size),0)*tile_size)
    cropy = (round(int(y/tile_size),0)*tile_size)
    startx = x//2-(cropx//2)
    starty=y//2-(cropy//2)
    print(startx, startx+cropx, starty, starty+cropy)
    window = Window(startx,starty,cropx,cropy)
    return window


## Set Paths and Tile Size

In [36]:
#point to original image file with the geo metadata intact
geo_path = r'C:/Users/Path_to_image_with_georef.tif'
data_path = r'C:/Users/Path_to_inference_output.tif'

#define the name of output image
output_path = r'C:/Users/Path_to_output.tif' 

tile_size = 128 #enter tile size used on inference
is_FSD = False #enter True if you are referencing a FSD map, enter False if you are referencing an activation heatmap

## Load Images, Set dtype to match Inference Image

In [37]:
raster_with_ref= rasterio.open(geo_path)

In [38]:
input_crs = raster_with_ref.crs
input_gt  = raster_with_ref.transform

print(input_crs)
print("image width: ",raster_with_ref.width,"  image height: ",raster_with_ref.height)
print("Upper Left Image Origin= X:",input_gt[2],", Y:",input_gt[5],"Pixel size: ", input_gt[0],"m,",input_gt[4],"m")

EPSG:2956
image width:  7983   image height:  5421
Upper Left Image Origin= X: 480586.64771000005 , Y: 6246389.955510001 Pixel size:  0.09999999999999855 m, -0.09999999999993128 m


In [39]:
ras = get_window_crop(raster_with_ref,tile_size)
print(ras.width,ras.height)

23 7959 22 5398
7936 5376


In [40]:
newxo = input_gt[2]+(((raster_with_ref.width-ras.width)/2)*input_gt[0])
newyo = input_gt[5]+(((raster_with_ref.height-ras.height)/2)*input_gt[4])
print(newxo,newyo)

480588.99771 6246387.705510001


In [41]:
if is_FSD == True:
    ground_resolution = tile_size/10
else:
    ground_resolution = input_gt[0]

## Rescale image to Prediction Map Resolution (e.g. 12.8m x 12.8m pixels)

In [42]:
rescale = rasterio.transform.from_origin(newxo, newyo,ground_resolution,ground_resolution)
print(rescale)

| 0.10, 0.00, 480589.00|
| 0.00,-0.10, 6246387.71|
| 0.00, 0.00, 1.00|


## Open Raster with Data (Inference Image)

In [44]:
raster_with_data = rasterio.open(data_path)

raster_array_with_data = raster_with_data.read()
dtype=raster_array_with_data.dtype

## Get Data Dimensions

In [45]:
print("image width: ",raster_with_data.width,"  image height: ",raster_with_data.height)
height = raster_with_data.height
width = raster_with_data.width

image width:  7983   image height:  5421


## Write Metadata to Inference Image

In [46]:
if is_FSD == True:
    
    with raster_with_ref as src:
        kwargs = raster_with_ref.meta.copy()
        print(kwargs)
    
        kwargs.update({
            'height': raster_with_data.height,
            'width': raster_with_data.width,
            'dtype': dtype,
            'count':raster_with_data.count,
            'transform':rescale})
        print(kwargs)
    
        with rasterio.open(output_path, 'w', **kwargs) as dst: #defining output image and referencing updated kwargs
            dst.write(raster_with_data.read())
            
else:
    with raster_with_ref as src:
        kwargs = raster_with_ref.meta.copy()
        print(kwargs)
    
        kwargs.update({
            'height': raster_with_data.height,
            'width': raster_with_data.width,
            'dtype': dtype,
            'count':raster_with_data.count,
            'transform':src.transform})
        print(kwargs)
    
        with rasterio.open(output_path, 'w', **kwargs) as dst: #defining output image and referencing updated kwargs
            dst.write(raster_with_data.read())

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 7983, 'height': 5421, 'count': 4, 'crs': CRS.from_epsg(2956), 'transform': Affine(0.09999999999999855, 0.0, 480586.64771000005,
       0.0, -0.09999999999993128, 6246389.955510001)}
{'driver': 'GTiff', 'dtype': dtype('float32'), 'nodata': None, 'width': 7983, 'height': 5421, 'count': 1, 'crs': CRS.from_epsg(2956), 'transform': Affine(0.09999999999999855, 0.0, 480586.64771000005,
       0.0, -0.09999999999993128, 6246389.955510001)}
